In [96]:
import numpy as np
import tensorflow as tf
import keras
import keras.layers as layers
import pickle
import wandb
from wandb.keras import WandbCallback
import time

In [35]:
glove_mat = np.load("./data/glove_6B_50d.npy")
words = np.load("./data/words_6B_50d.npy")
with open("./data/word_to_ind_6B_50d.pkl", "rb") as f:
    word_to_ind = pickle.load(f)

In [36]:
X_train = np.load("./data/google_mikolov/X_train.npy")
X_dev = np.load("./data/google_mikolov/X_dev.npy")
X_test = np.load("./data/google_mikolov/X_test.npy")
X_test_words = np.load("./data/google_mikolov/X_test_words.npy")

Y_train = np.load("./data/google_mikolov/Y_train.npy")
Y_dev = np.load("./data/google_mikolov/Y_dev.npy")
Y_test = np.load("./data/google_mikolov/Y_test.npy")
Y_test_words = np.load("./data/google_mikolov/Y_test_words.npy")

In [48]:
def dist(a, b):
    return np.linalg.norm(glove_mat[word_to_ind[a]] - glove_mat[word_to_ind[b]])

def closest_neighbors(embedding, n):
    neighbors = sorted(words, key=lambda word: np.linalg.norm(embedding - glove_mat[word_to_ind[word]]))
    return neighbors[:n]

In [128]:
delta = 1.0
h = 1.0

def mse_large(y_true, y_pred):
    mse = keras.losses.mse(y_true, y_pred)
    # penalty = tf.divide(delta, tf.math.add(h, tf.norm(y_pred))) 
    penalty = tf.maximum(0.0, tf.multiply(-0.5/5.0, tf.subtract(tf.norm(y_pred), 5.0)))
    # print(mse, penalty)
    return tf.math.add(mse, penalty)

print(mse_large(minnesota, bulletinyyy))
print(mse_large(florida, instance))

tf.Tensor(1.0125517529035342, shape=(), dtype=float64)
tf.Tensor(0.5664634466615177, shape=(), dtype=float64)


In [122]:
config = {
    "architecture": "dense",
    "dataset": "google_mikolov",
    "optimizer": "adam",
    "loss": "mse_large",
    "metrics": ["mse"],
    "layers": 3,
    "hidden_layers": [300, 300]
}

inputs = layers.Input(shape=(150))
X = inputs

assert(config["layers"] == len(config["hidden_layers"]) + 1)

for size in config["hidden_layers"]:
    X = layers.Dense(size, activation="relu")(X)
    X = layers.Dropout(0.2)(X)

Y = layers.Dense(50, activation="relu")(X)

model = keras.Model(inputs=inputs, outputs=Y, name="analogy_model")
model.summary()


Model: "analogy_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 150)]             0         
_________________________________________________________________
dense_58 (Dense)             (None, 300)               45300     
_________________________________________________________________
dropout_41 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_42 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_60 (Dense)             (None, 50)                15050     
Total params: 150,650
Trainable params: 150,650
Non-trainable params: 0
_______________________________________________

In [129]:
run = wandb.init(project="word-analogies", name=config["architecture"] + "-" + str(config["layers"]) + "-" + str(config["hidden_layers"]), config=config, notes="tweaked mse_large")

model.compile(loss=mse_large, optimizer=config["optimizer"], metrics=config["metrics"])
model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_data=(X_dev, Y_dev), callbacks=[WandbCallback()])

run.finish()

if input("Save model? (y/n): ") == "y":
    model.save("models/" + config["architecture"] + "-" + str(config["layers"]) + "-" + str(config["hidden_layers"]) + "-" + str(round(time.time())))

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch 1/100
489/489 [==============================] - 1s 3ms/step - loss: 0.3951 - mse: 0.3951 - val_loss: 0.2486 - val_mse: 0.2486
Epoch 2/100
489/489 [==============================] - 1s 2ms/step - loss: 0.2660 - mse: 0.2660 - val_loss: 0.2427 - val_mse: 0.2426
Epoch 3/100
489/489 [==============================] - 1s 2ms/step - loss: 0.2590 - mse: 0.2590 - val_loss: 0.2406 - val_mse: 0.2406
Epoch 4/100
489/489 [==============================] - 1s 2ms/step - loss: 0.2556 - mse: 0.2556 - val_loss: 0.2396 - val_mse: 0.2396
Epoch 5/100
489/489 [==============================] - 1s 3ms/step - loss: 0.2535 - mse: 0.2535 - val_loss: 0.2390 - val_mse: 0.2390
Epoch 6/100
489/489 [==============================] - 1s 3ms/step - loss: 0.2520 - mse: 0.2520 - val_loss: 0.2387 - val_mse: 0.2386
Epoch 7/100
489/489 [==============================] - 1s 2ms/step - loss: 0.2509 - mse: 0.2509 - val_loss: 0.2380 - val_mse: 0.2380
Epoch 8/100
489/489 [==============================] - 1s 2ms/step - 

epoch,99
loss,0.24517
mse,0.24517
val_loss,0.23562
val_mse,0.23562
_step,99
_runtime,98
_timestamp,1602227352
best_val_loss,0.2355
best_epoch,95


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


INFO:tensorflow:Assets written to: models/dense-3-[300, 300]-1602227589/assets


In [130]:
model.evaluate(X_test, Y_test, verbose=2)
Y_predict = model.predict(X_test)
Y_predict.shape

62/62 - 0s - loss: 0.2358 - mse: 0.2358


(1955, 50)

In [131]:
indices = np.random.randint(Y_predict.shape[0], size=10)
for i, emb in enumerate(Y_predict[indices]):
    print(Y_test_words[indices[i]], closest_neighbors(emb, 5), "Q:", X_test_words[indices[i]])

Zimbabwe ['zimbabwe', 'bringing', 'brings', 'namely', 'besides'] Q: ['Banjul' 'Gambia' 'Harare']
swiftly ['likewise', 'bulletinyyy', 'meantime', 'srivalo', 'piyanart'] Q: ['serious' 'seriously' 'swift']
describes ['explains', 'describes', 'introduces', 'wherein', 'relates'] Q: ['sit' 'sits' 'describe']
unpleasant ['unpleasant', 'irritating', 'comforting', 'fortunately', 'avoids'] Q: ['fortunate' 'unfortunate' 'pleasant']
Greenland ['greenland', 'greenlandic', 'klerksdorp', 'lillestroem', 'alunite'] Q: ['Madrid' 'Spain' 'Nuuk']
Russian ['counterparts', 'russian', 'besides', 'specialists', 'adding'] Q: ['Portugal' 'Portuguese' 'Russia']
slowing ['coupled', 'steady', 'keeps', 'lag', 'tends'] Q: ['dance' 'dancing' 'slow']
Australian ['besides', 'whereas', 'adding', 'bringing', 'additionally'] Q: ['Germany' 'German' 'Australia']
sitting ['sitting', 'stays', 'resting', 'basically', 'fills'] Q: ['say' 'saying' 'sit']
Israeli ['adding', 'essential', 'sources', 'besides', 'bringing'] Q: ['Thail

In [108]:
minnesota = glove_mat[word_to_ind["minnesota"]]
bulletinyyy = glove_mat[word_to_ind["bulletinyyy"]]
florida = glove_mat[word_to_ind["florida"]]
instance = glove_mat[word_to_ind["instance"]]

In [107]:
print(np.linalg.norm(glove_mat[word_to_ind["besides"]]))
print(np.linalg.norm(glove_mat[word_to_ind["instance"]]))
print(np.linalg.norm(glove_mat[word_to_ind["whereas"]]))
print(np.linalg.norm(glove_mat[word_to_ind["serves"]]))
print(np.linalg.norm(glove_mat[word_to_ind["bulletinyyy"]]))
print(np.linalg.norm(glove_mat[word_to_ind["tunisia"]]))

3.1332464376507314
3.4258391633512217
3.3534548890073057
4.144025282606665
0.05465219281056251
5.059411990967187
